In [1]:
from langdetect import detect
import pandas as pd
from sklearn.metrics import accuracy_score
import numpy as np
import os
import time

In [2]:
df = pd.read_csv("../../data/qald-1-test.csv")

In [6]:
df= df.dropna(subset=['questions'])

In [7]:
with open('opennlp_input.txt', 'w') as f:
    for q in df["questions"]:
        f.write(q+"\n\n")

In [8]:
# Run openNLP Language Detector
import datetime
a = time.time()
os.system("/home/oshara/GSoC/DBPedia/openNLP/apache-opennlp-1.9.2/bin/opennlp LanguageDetector /home/oshara/GSoC/DBPedia/openNLP/langdetect-183.bin </home/oshara/GSoC/DBPedia/QLang/evaluation/opennlp/opennlp_input.txt> /home/oshara/GSoC/DBPedia/QLang/evaluation/opennlp/opennlp_output.txt")
b = time.time()
c = b-a
c/df.shape[0]

0.0006802016468139356

In [9]:
result=[]
count = 0
def extractLan(f):
    global result
    global count
    while count < df.shape[0]:
        line = f.readline()
        if line == "\n":
            continue
        result.append(line[:3])
        count += 1
        
with open('opennlp_output.txt', 'r') as f:
    extractLan(f)

In [10]:
df["lang"].unique()

array(['de', 'ru', 'pt', 'en', 'hi_IN', 'fa', 'pt_BR', 'it', 'fr', 'ro',
       'es', 'nl'], dtype=object)

In [11]:
# languages
# q3 ['en', 'de', 'es', 'it', 'fr', 'nl']
# q4 ['en', 'de', 'es', 'it', 'fr', 'nl', 'ro']
# q5 ['en', 'de', 'es', 'it', 'fr', 'nl', 'ro']
# q6 ['en', 'fa', 'de', 'es', 'it', 'fr', 'nl', 'ro']
# q7 ['en', 'de', 'it', 'fr']
# q8 ['en']

In [12]:
# English, Spanish, German, Italian, French, Dutch, Romanian, fas
# {'en': 0, 'de': 1, 'es': 2, 'it': 3, 'fr': 4, 'nl': 5, 'ro': 6}

# q3,q4,q5
# label = {'eng': 0, "spa": 2, "deu": 1, "ita": 3, "fra": 4, "nld": 5, "ron": 6, "other": 7}

# q6
# label = {'eng': 0,"fas":1 ,"spa": 3, "deu": 2, "ita": 4, "fra": 5, "nld": 6, "ron": 7, "other": 8}
# q7,q8
# label = {'eng': 0, "deu": 1, "ita": 2, "fra": 3,"other": 4}

# q9
label = {"deu":0,"rus":1,"por":2,"eng":3,"hin":4,"fas":5,"por":6,"ita":7,"fra":8,"ron":9,"spa":10,"nld":11,"other":12}

In [13]:
for i in range(len(result)):
    key = result[i]
    if key in label:
        result[i] = label[key]
    else:
        result[i] = -1

In [14]:
possible_labels = df["lang"].unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

df['lang'] = df.lang.replace(label_dict)

In [15]:
y_true = df['lang']

In [17]:
accuracy_score(y_true, result)

0.4138755980861244